<a href="https://colab.research.google.com/github/RaminTK/PATH/blob/main/PATH06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
import nltk
nltk.download('punkt')
nltk.download('stopwords')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
data = pd.read_csv('drive/MyDrive/path/PATHAlerts.csv')
data = data.drop('Video Thumbnail', axis=1)
data = data.drop('Image', axis=1)
data.fillna(0, inplace=True)
data['Content'][2]

'JSQ-33 via HOB resuming normal service. Earlier smoke condition resulting in 3rd rail power loss has concluded at CHRS St.  We apologize for the inconvenience this may have caused.'

In [ ]:
def preprocess_text(text):
    # Remove special characters
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])

    # Convert to lowercase
    text = text.lower()

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    return ' '.join(tokens)

data['Processed_Tweet'] = data['Content'].apply(preprocess_text)


In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['Processed_Tweet'])
feature_names = vectorizer.get_feature_names_out()


In [ ]:
print("Most Frequent Words (Delay Reasons):")
for word, freq in word_freq_list:
    print(f"{word}: {freq} occurrence(s)")


Most Frequent Words (Delay Reasons):
the: 650 occurrence(s)
at: 610 occurrence(s)
st: 513 occurrence(s)
an: 484 occurrence(s)
for: 481 occurrence(s)
in: 475 occurrence(s)
update: 458 occurrence(s)
will: 445 occurrence(s)
we: 428 occurrence(s)
be: 395 occurrence(s)
15: 391 occurrence(s)
mins: 388 occurrence(s)
approx: 387 occurrence(s)
issued: 380 occurrence(s)
service: 375 occurrence(s)
pm: 368 occurrence(s)
to: 355 occurrence(s)
inconvenience: 339 occurrence(s)
nwkwtc: 313 occurrence(s)
jsq33: 299 occurrence(s)
apologize: 294 occurrence(s)
this: 268 occurrence(s)
delayed: 267 occurrence(s)
and: 254 occurrence(s)
train: 246 occurrence(s)
hob: 234 occurrence(s)
from: 233 occurrence(s)
delays: 230 occurrence(s)
have: 223 occurrence(s)
normal: 215 occurrence(s)
may: 206 occurrence(s)
caused: 204 occurrence(s)
of: 198 occurrence(s)
problem: 197 occurrence(s)
earlier: 196 occurrence(s)
resuming: 191 occurrence(s)
is: 182 occurrence(s)
am: 181 occurrence(s)
continue: 178 occurrence(s)
crew: 

In [ ]:
word_freq = X.sum(axis=0)
word_freq_list = [(word, word_freq[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
word_freq_list.sort(key=lambda x: x[1], reverse=True)


In [ ]:
delay_reasons = []
delay_durations = []
for tweet in data['Content']:
    # Extract delay reason
    delay_reason_matches = re.findall(delay_reason_regex, tweet, re.IGNORECASE)
    if delay_reason_matches:
        delay_reasons.extend(delay_reason_matches)

    # Extract delay duration
    delay_duration_matches = re.findall(delay_duration_regex, tweet, re.IGNORECASE)
    if delay_duration_matches:
        delay_durations.extend(delay_duration_matches)


In [ ]:
delay_reason_counts = Counter(delay_reasons)
delay_reason_counts

Counter({('continues.', '', ' '): 2,
         ('continues', '', ' '): 1,
         ('is', '', ' '): 1,
         ('was', '', ' '): 1,
         ('at 9th', '', ' '): 1})

In [ ]:
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z]', ' ', text)

    # Convert to lowercase
    text = text.lower()

    # Remove stopwords (if required, you may need to download stopwords using nltk.download('stopwords'))
    # from nltk.corpus import stopwords
    # stopwords = set(stopwords.words('english'))
    # text = ' '.join(word for word in text.split() if word not in stopwords)

    return text

# Apply the preprocessing function to the 'Content' column
data['Content'] = data['Content'].apply(preprocess_text)
data['Content']

0      there s a red bull arena event sun            ...
1      trains from newark   harrison are once again d...
2      jsq    via hob resuming normal service  earlie...
3      jsq    via hob delayed  crew investigating smo...
4            pm  jsq    via hob delays continue  main...
                             ...                        
934    jsq    resuming normal service  papd resolved ...
935    nwk wtc operating with residual delays from an...
936    jsq    delays continue  hob    resuming normal...
937    nwk wtc operating with  residual delays from a...
938    jsq     hob    delayed  crew reported a trespa...
Name: Content, Length: 939, dtype: object

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['Content'])
X

<939x587 sparse matrix of type '<class 'numpy.int64'>'
	with 20282 stored elements in Compressed Sparse Row format>

In [ ]:
kmeans = KMeans(n_clusters=5)  # You can adjust the number of clusters as per your needs
kmeans.fit(X)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=5)

In [ ]:
clusters = kmeans.predict(X)

for cluster_id in range(kmeans.n_clusters):
    cluster_tweets = data[clusters == cluster_id]['Content']
    print(f"Cluster {cluster_id}:")
    print(cluster_tweets)
    print()


Cluster 0:
25       st and    st stations closed nightly from ap...
34       st and    st stations closed nightly from ap...
45       st and    st stations closed nightly from ap...
70         st      st      st and    st stations  usu...
78         st      st      st and    st stations  usu...
                             ...                        
898      st and    st stations closed nightly from ap...
912      st and    st stations closed nightly from ap...
925      st and    st stations closed nightly from ap...
926      st and    st stations closed nightly from ap...
931      st and    st stations closed nightly from ap...
Name: Content, Length: 70, dtype: object

Cluster 1:
3      jsq    via hob delayed  crew investigating smo...
4            pm  jsq    via hob delays continue  main...
5            pm  jsq    via hob delayed  dispatch has...
9      nwk wtc delayed  crews working to lock hack ri...
10     nwk wtc delayed  crews continue working to loc...
                        

In [ ]:
import tweepy

# Enter your own Twitter API credentials here
consumer_key = 'API_KEY'
consumer_secret = 'API_SECRET_KEY'
access_token = 'ACCESS_TOKEN'
access_token_secret = 'ACCESS_TOKEN_SECRET'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

# You need to replace 'PATHTrain' with the actual Twitter handle of NJ PATH
tweets = api.user_timeline(screen_name='PATHTrain', count=200, tweet_mode='extended')

disruptions = ['delay', 'disruption', 'maintenance', 'suspended', 'cancelled']

for tweet in tweets:
    if any(disruption in tweet.full_text.lower() for disruption in disruptions):
        print(f"{tweet.created_at} - {tweet.full_text}\n")


Unauthorized: ignored